In [1]:
import requests

In [2]:
url = "https://www.worldometers.info/coronavirus"

In [3]:
req = requests.get(url)

In [4]:
req

<Response [200]>

In [5]:
from lxml.html import fromstring

In [6]:
root = fromstring(req.text)

In [7]:
table = root.xpath(".//table[@id='main_table_countries_today'][1]")[0]

In [8]:
table

<Element table at 0x7efbe838aa70>

In [9]:
thead = table.xpath(".//thead/tr/th")

In [10]:
def is_visible(attr):
    if 'style' in attr.attrib:
        style = attr.get('style')
        if style.__contains__("display:none") or style.__contains__("display: none"):
            return False
    return True

In [11]:
columns = list()

In [12]:
for th in thead:
    if is_visible(th) is True:
        columns.append(th.text_content().replace('\n', '').replace(u'\xa0', u'').strip())

In [13]:
columns

['#',
 'Country,Other',
 'TotalCases',
 'NewCases',
 'TotalDeaths',
 'NewDeaths',
 'TotalRecovered',
 'ActiveCases',
 'Serious,Critical',
 'TotCases/1M pop',
 'Deaths/1M pop',
 'TotalTests',
 'Tests/1M pop',
 'Population']

In [14]:
tbody = table.xpath(".//tbody/tr")

In [15]:
tags = list()
values = list()

In [16]:
for tr in tbody:
    if is_visible(tr) is True:
        tag = tr.xpath(".//td/a")
        if len(tag) > 0:
            tags.append(tag[0].get('href').strip().replace('country/', '').replace('/', ''))

        values.append([value.text_content().strip() for value in tr.xpath(".//td") if is_visible(value) is True])

In [17]:
import pandas as pd

In [18]:
data = pd.DataFrame(values, columns=columns)

In [19]:
data.head()

,#,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,ActiveCases,"Serious,Critical",TotCases/1M pop,Deaths/1M pop,TotalTests,Tests/1M pop,Population
0,,World,"6,091,690","+65,272","368,704","+2,282","2,698,600","3,024,386","53,549",782,47.3,,,
1,1,USA,"1,805,270","+11,740","104,977",+435,"521,016","1,179,277","17,205","5,457",317,"17,017,473","51,438","330,832,890"
2,2,Brazil,"469,510","+1,172","28,015",+71,"193,181","248,314","8,318","2,210",132,"930,013","4,378","212,426,274"
3,3,Russia,"396,575","+8,952","4,555",+181,"167,469","224,551","2,300","2,718",31,"10,316,200","70,693","145,929,167"
4,4,Spain,"286,308",+664,"27,125",+4,"196,958","62,225",617,"6,124",580,"3,556,567","76,071","46,753,246"


In [20]:
data.to_csv('../data/overview.csv')